In [2]:
from scapy.all import *
from pprint import pprint
import sys
import numpy as np
import os
import dpkt

KEYLEN=8

VALUE_PKT_SIZE = 591
GET_PKT_SIZE = 64
MAX_LINE_RATE =10e9


In [3]:
def read_pcap_with_pkt(out_dir, dst_mac_is_ts = True, try_compare_counters = True):
    
    if try_compare_counters:
        count_file = os.path.join(out_dir, 'counters_0.out')
        for line in open(count_file):
            if line.startswith('7/0'):
                print("counters_0.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_start_pkts = int(n_pkts_str[:-1])
        count_file = os.path.join(out_dir, 'counters_1.out')
        for line in open(count_file):
            if line.startswith('7/0'):
                print("counters_1.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_end_pkts = int(n_pkts_str[:-1])   
        n_expected_pkts = (n_end_pkts - n_start_pkts)
        
        print("Expecting {} packets".format(n_expected_pkts))
    
    
    pcf = os.path.join(out_dir, 'moongen', 'moongen.pcap')
    
    counters = {
        'other': 0,
        'packet': 0,
    }

    f = open(pcf, 'rb')
    
    print("Reading {}".format(pcf))
    pcap=dpkt.pcap.Reader(f)
    total_size = 0
    pkt_ingress = []
    pkt_egress = []
    pkt_throughput = []
    all_packets = []
    
           
    f = open(pcf, 'rb')
    print("Reading {}".format(pcf))
    pcap=dpkt.pcap.Reader(f)
    
    for ts, buf in pcap:
        try:
            eth=dpkt.ethernet.Ethernet(buf)
            ip=eth.data
            
            #TODO: Add a check to distinguish ingress and egress packets
            # If ingress, add to input array.
            # If egress, add to output array
            if ((eth.type != 0x0800) and (eth.type != 0x86dd) and (eth.type != 0x88cc)):
                y3, y2, y1 = struct.unpack("!HHH", eth.src)
                if(y3 == 0x00 and y2 == 0x00 and y1 == 0x01):
                    counters['other'] += 1
                else:
                    continue
            
        except Exception as e:
            print("Exception parsing packet {} : {}".format(buf, e))
            Ether(buf).show2()
            raise
        
        if dst_mac_is_ts:
            x3, x2, x1 = struct.unpack("!HHH", eth.dst)
    
            if (counters['other'] == 1):
                ts_init = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
                #ts = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
                pkt_throughput.append(ts_init)
           
                    
            ts_last = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
            #ts = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
            
    
        pkt_throughput.append(ts_last)

    print(pkt_throughput)
    #all_packets.append(pkt_ingress)
    #all_packets.append(pkt_egress)
    print(counters['other'])
    #return lists of arrays for ingress and egress packets.
    #return all_packets
    length = len(pkt_throughput)
    time_elapsed = pkt_throughput[length - 2] - pkt_throughput[2]
    
    print(time_elapsed)

    #return 1

In [11]:
def read_pcap(out_dir, dst_mac_is_ts = True, try_compare_counters = True):
    
    if try_compare_counters:
        count_file = os.path.join(out_dir, 'counters_0.out')
        for line in open(count_file):
            if line.startswith('7/0'):
                print("counters_0.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_start_pkts = int(n_pkts_str[:-1])
        count_file = os.path.join(out_dir, 'counters_1.out')
        for line in open(count_file):
            if line.startswith('7/0'):
                print("counters_1.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_end_pkts = int(n_pkts_str[:-1])   
        n_expected_pkts = (n_end_pkts - n_start_pkts)
        
        print("Expecting {} packets".format(n_expected_pkts))
    
    
    pcf = os.path.join(out_dir, 'moongen', 'moongen.pcap')
    
    counters = {
        'other': 0,
        'packet': 0,
    }

    f = open(pcf, 'rb')
    
    print("Reading {}".format(pcf))
    pcap=dpkt.pcap.Reader(f)
    total_size = 0
    pkt_ingress = []
    pkt_egress = []
    pkt_throughput = []
    all_packets = []
    
           
    f = open(pcf, 'rb')
    print("Reading {}".format(pcf))
    pcap=dpkt.pcap.Reader(f)
    
    for ts, buf in pcap:
        try:
            eth=dpkt.ethernet.Ethernet(buf)
            ip=eth.data
            
            #TODO: Add a check to distinguish ingress and egress packets
            # If ingress, add to input array.
            # If egress, add to output array
            if ((eth.type != 0x0800) and (eth.type != 0x86dd) and (eth.type != 0x88cc)):
                y3, y2, y1 = struct.unpack("!HHH", eth.src)
                if(y3 == 0x00 and y2 == 0x00 and y1 == 0x01):
                    counters['other'] += 1
                else:
                    continue
            
        except Exception as e:
            print("Exception parsing packet {} : {}".format(buf, e))
            Ether(buf).show2()
            raise
        
        if dst_mac_is_ts:
            x3, x2, x1 = struct.unpack("!HHH", eth.dst)
    
            if (counters['other'] == 1):
                ts_init = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
                #ts = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
                pkt_throughput.append(ts_init)
           
                    
            ts_last = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
            #ts = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
            
    
        pkt_throughput.append(ts_last)

    #print(pkt_throughput)
    #all_packets.append(pkt_ingress)
    #all_packets.append(pkt_egress)
    print(counters['other'])
    #return lists of arrays for ingress and egress packets.
    #return all_packets
    length = len(pkt_throughput)
    time_elapsed = pkt_throughput[length - 1] - pkt_throughput[0]
    
    print(time_elapsed)

    #return 1

In [12]:
def get_latency(out_dir, dst_mac_is_ts=True):
    
    pcf = os.path.join(out_dir, 'moongen', 'moongen.pcap')
    
    counters = {
        'ingress': 0,
        'egress': 0,
    }

    f = open(pcf, 'rb')
    
    print("Reading {}".format(pcf))
    pcap=dpkt.pcap.Reader(f)
    total_size = 0
    pkt_ingress = []
    pkt_egress = []
    pkt_latency = []
    
    for ts, buf in pcap:
        try:
            eth=dpkt.ethernet.Ethernet(buf)
            
            #ip=eth.data
            
            #TODO: Add a check to distinguish ingress and egress packets
            # If ingress, add to input array.
            # If egress, add to output array
            
            if ((eth.type != 0x86dd) and (eth.type != 0x88cc)):
                y3, y2, y1 = struct.unpack("!HHH", eth.src)
                if (y3 == 0x00 and y2 == 0x00 and y1 == 0x01):
                    counters['egress'] += 1
                elif (y3 == 0x00 and y2 == 0x00 and y1 == 0x02):    
                    counters['ingress'] += 1
            else:
                continue
            
        except Exception as e:
            print("Exception parsing packet {} : {}".format(buf, e))
            Ether(buf).show2()
            raise

        if dst_mac_is_ts:
            x3, x2, x1 = struct.unpack("!HHH", eth.dst)
            ts = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
            
            y3, y2, y1 = struct.unpack("!HHH", eth.src)
            if(y3 == 0x00 and y2 == 0x00 and y1 == 0x01):
                pkt_egress.append(ts)
            elif (y3 == 0x00 and y2 == 0x00 and y1 == 0x02):    
                pkt_ingress.append(ts)
     
    print(counters['ingress'])
    print(counters['egress']) 
    
    length = len(pkt_ingress)
    i = 0
    while (i < length):
        pkt_latency.append(pkt_egress[i] - pkt_ingress[i])
        i += 1
        
    i = 0
    tot = 0
    
    
    
    for val in pkt_latency:
        tot += val
    
    print(tot)
    print(length)
    avg = tot / length

    print(avg)
    #return avg
   

In [13]:
out_dir = "/home/ncshy/P4Boosters/testing/fec/execution/test_output/fpga_fec_encoder/fpga_fec_encoder_75.00/"
#read_pcap_with_pkt(out_dir, True, True)
read_pcap(out_dir, True, True)
#get_latency(out_dir, True)

counters_0.out successfully opened
counters_1.out successfully opened
Expecting 2200 packets
Reading /home/ncshy/P4Boosters/testing/fec/execution/test_output/fpga_fec_encoder/fpga_fec_encoder_75.00/moongen/moongen.pcap
Reading /home/ncshy/P4Boosters/testing/fec/execution/test_output/fpga_fec_encoder/fpga_fec_encoder_75.00/moongen/moongen.pcap
1200
0.0011343830000001276
